In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import sys
sys.path.append('../src')


from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation
from sklearn.metrics import roc_auc_score

In [3]:
SEED = 33
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [4]:
# load adult
kdd_df = pd.read_csv('../datasets/proc/crafted_obesity.csv')

y_pv = kdd_df[['target','Gender']]
X = kdd_df.drop(columns=['target', 'Gender'])

X_train, X_test, y_pv_train, y_pv_test = train_test_split(X, y_pv, test_size=0.3, shuffle=True)

pv_test = y_pv_test['Gender']
pv_train = y_pv_train['Gender']
y_train = y_pv_train['target']
y_test = y_pv_test['target']

FairOD on adult dataset

In [5]:
model = OutlierDetector(alpha=0.1, gamma=0.8)
train_hist = model.fit(X_train, pv_train, batch_size=512, epochs=6), #val_X=X_test, val_pv=pv_test)

Predict

In [6]:
# predict
X_pred = model.predict_scores(X_test)

# compute reconstruction error of predicted samples and save in dataframe
#outlier_score = evalu.compute_outlier_scores(X_test, X_pred)
outlier_score = X_pred
pred_df = pd.DataFrame({'Reconstruction_error':outlier_score, 'True_class': y_test})

Evaluate

In [7]:
print("AUC score:   ", roc_auc_score(y_test, X_pred))
print("AUC RATIO:   ", evaluation.auc_ratio(y_test, X_pred, pv_test))
# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, outlier_score, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, outlier_score, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, outlier_score, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(outlier_score, pv_test).numpy())

AUC score:    0.6784723355638975
AUC RATIO:    0.756890299164537
AP RATIO  1.167192529864243
PRECISION RATIO  1.2380952380952381
FAIRNESS METRIC   0.6361904761904762
GROUP FIDELITY METRIC   1.1336668


## SG-AE Method

In [8]:
from fairsgod.losses import FairODLoss
from fairsgod.fairod import OutlierDetector
from fairsgod.fairod_sg import SGOutlierDetector
from fairsgod import evaluation

In [9]:
SEED = 33
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [43]:
model = SGOutlierDetector(epsilon=0.001, 
                          lambda_se=4.86, 
                          lambda_a=25.0,
                          a=20, 
                          alpha=0.01, 
                          gamma=0.25)

_, _ = model.fit(X_train, pv_train, batch_size=512, epochs=10, val_X=None, val_pv=None)

In [44]:
# save predictions
X_pred = model.predict_scores(X_test).numpy()

In [45]:
print("AUC score:   ", roc_auc_score(y_test, X_pred))
print("AUC RATIO:   ", evaluation.auc_ratio(y_test, X_pred, pv_test))
# AP ratio
print("AP RATIO ", evaluation.compute_AP_ratio(y_test, X_pred, pv_test))
# Precision ratio
print("PRECISION RATIO ", evaluation.compute_precision_ratio(y_test, X_pred, pv_test))
# Fairness metric
print("FAIRNESS METRIC  ", evaluation.compute_Fairness_metric(y_test, X_pred, pv_test).numpy())
# Group Fidelity metric
print("GROUP FIDELITY METRIC  ", evaluation.compute_GF_metric(X_pred, pv_test).numpy())

AUC score:    0.7426379724404859
AUC RATIO:    0.9252244177779029
AP RATIO  0.8161633208332559
PRECISION RATIO  0.6518218623481781
FAIRNESS METRIC   0.8479934540856177
GROUP FIDELITY METRIC   0.0037615204
